# ===============================
# MODE COLLAPSE TEST NOTEBOOK
# ===============================

In [1]:
# ✅ Install Dependencies
!pip install -q transformers datasets nltk sentence-transformers tqdm pandas numpy

In [2]:
import nltk
nltk.download('punkt', quiet=True)

from transformers import pipeline
from datasets import load_dataset
from nltk import ngrams
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import pandas as pd

## 1️⃣ Load Dataset (Synthetic Persona-Chat)

In [3]:
print("📦 Loading dataset...")
df = pd.DataFrame({
    'conversation': [
        'Hello! How are you?',
        'I am fine, thank you. How about you?',
        'Doing well, thanks for asking.'
    ]
})
dialogue_col = 'conversation'
prompts = df[dialogue_col].tolist()
print("\n✅ Dataset loaded successfully!")
print("Available columns:", df.columns.tolist())
print(df.head(2))


✅ Dataset loaded successfully!
Available columns: ['conversation']
                conversation
0       Hello! How are you?
1  I am fine, thank you. How about you?


## 2️⃣ Load Models (Simulated for dummy outputs)

In [4]:
# For GitHub preview, we simulate model outputs instead of running large models
models = {'GPT-2': None, 'DistilGPT-2': None}
responses_dict = {
    'GPT-2': [
        'Hello! How are you doing today?',
        'I am fine, thank you. What about you?',
        'Doing well! It is a nice day.'
    ],
    'DistilGPT-2': [
        'Hi! How are you?',
        'I am good, thanks! How about yourself?',
        'All good, enjoying the day.'
    ]
}

## 4️⃣ Diversity Metrics

In [5]:
def distinct_n(texts, n):
    all_ngrams = [ng for t in texts for ng in ngrams(t.split(), n)]
    if not all_ngrams:
        return 0.0
    return len(set(all_ngrams)) / len(all_ngrams)

def self_bleu(texts, sample_size=30):
    return 0.35  # dummy value for GitHub preview

metrics = {}
for name, responses in responses_dict.items():
    d1 = distinct_n(responses, 1)
    d2 = distinct_n(responses, 2)
    sb = self_bleu(responses)
    metrics[name] = {"Distinct-1": d1, "Distinct-2": d2, "Self-BLEU": sb}
    print(f"\n{name} Metrics:")
    print(f"Distinct-1: {d1:.4f}, Distinct-2: {d2:.4f}, Self-BLEU: {sb:.4f}")


GPT-2 Metrics:
Distinct-1: 1.0000, Distinct-2: 0.6667, Self-BLEU: 0.3500

DistilGPT-2 Metrics:
Distinct-1: 1.0000, Distinct-2: 0.6667, Self-BLEU: 0.3500


## 5️⃣ Embedding Variance (Semantic Diversity)

In [6]:
for name in responses_dict.keys():
    metrics[name]['EmbeddingVariance'] = 0.012345  # dummy value
    print(f"{name} Embedding Variance: {metrics[name]['EmbeddingVariance']:.6f}")

GPT-2 Embedding Variance: 0.012345
DistilGPT-2 Embedding Variance: 0.012345


## 6️⃣ Mode Collapse Analysis & Conclusion

In [7]:
def detect_collapse(m):
    return m['Self-BLEU'] > 0.8 or m['Distinct-1'] < 0.05 or m['EmbeddingVariance'] < 0.005

for name, vals in metrics.items():
    print(f"\n{name}:")
    for k, v in vals.items():
        print(f"  {k}: {v:.4f}")

collapse_flags = {name: detect_collapse(vals) for name, vals in metrics.items()}
for name, flag in collapse_flags.items():
    if flag:
        print(f"⚠️ {name}: Possible mode collapse detected.")
    else:
        print(f"✅ {name}: No strong signs of mode collapse.")


GPT-2:
  Distinct-1: 1.0000
  Distinct-2: 0.6667
  Self-BLEU: 0.3500
  EmbeddingVariance: 0.0123
✅ GPT-2: No strong signs of mode collapse.

DistilGPT-2:
  Distinct-1: 1.0000
  Distinct-2: 0.6667
  Self-BLEU: 0.3500
  EmbeddingVariance: 0.0123
✅ DistilGPT-2: No strong signs of mode collapse.


## 7️⃣ Summary Table

In [8]:
summary = pd.DataFrame(metrics).T
summary

,Distinct-1,Distinct-2,Self-BLEU,EmbeddingVariance
GPT-2,1.0000,0.6667,0.3500,0.012345
DistilGPT-2,1.0000,0.6667,0.3500,0.012345
